<a href="https://colab.research.google.com/github/lalit-jamdagnee/Learning_ML/blob/main/excercise_ensemble_learning_and_random_forests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Load the MNIST data and split it into a training set, a validation set and test set (e.g., use 50,000 instances for training, 10,000 for validation, and 10,000 for testing). Then train various classifiers, such as a random Forest classifer, an Extra-Trees classifier, and an SVM classifier.

In [6]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [7]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(mnist.data,
                                                            mnist.target,
                                                            test_size= 10000,
                                                            random_state=42)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val,
                                                  y_train_val,
                                                  test_size=10000,
                                                  random_state=42)

In [9]:
# importing all the models
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.svm import LinearSVC

In [17]:
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
ext_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(random_state=42,max_iter = 100)
voting_clf = VotingClassifier(
    estimators=[('rnd', rnd_clf), ('ext', ext_clf), ('svm',svm_clf)],
    voting='hard'
)

In [18]:
# Fitting the classifiers
estimators = [rnd_clf, ext_clf, svm_clf]

for estimator in estimators:
  print("Training the", estimator.__class__.__name__)
  estimator.fit(X_train, y_train)

Training the RandomForestClassifier
Training the ExtraTreesClassifier
Training the LinearSVC


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [19]:
# Scores of the estimators on validation set

for estimator in estimators:
  print(estimator.__class__.__name__, estimator.score(X_val, y_val))

RandomForestClassifier 0.9692
ExtraTreesClassifier 0.9715
LinearSVC 0.8556


Next, try to combine them into an ensemble that outperforms each individual classifier on the validation set, using soft or hard voting. Once you have found one, try it on the test set. How much better does it perform compared to the individual classifiers?

In [20]:
# First we will check using Hard voting
voting_clf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


VotingClassifier(estimators=[('rnd', RandomForestClassifier(random_state=42)),
                             ('ext', ExtraTreesClassifier(random_state=42)),
                             ('svm', LinearSVC(max_iter=100, random_state=42))])

In [21]:
# score on validation set
voting_clf.score(X_val, y_val)

0.9693

In [23]:
# Now check using soft voting
voting_clf2 = VotingClassifier(
    estimators=[('rnd', rnd_clf), ('ext', ext_clf)],
    voting='soft'
)

In [24]:
voting_clf2.fit(X_train, y_train)

VotingClassifier(estimators=[('rnd', RandomForestClassifier(random_state=42)),
                             ('ext', ExtraTreesClassifier(random_state=42))],
                 voting='soft')

In [25]:
print("validation score :", voting_clf2.score(X_val, y_val))

validation score : 0.9719


Now let's check the performance of each classifier on the test set:


In [26]:
from sklearn.metrics import accuracy_score as accuracy

scores = [] #stores the score of each classifier on test set
for estimator in estimators:
  y_pred = estimator.predict(X_test)
  score = "{}: {:.2f}".format(estimator.__class__.__name__, accuracy(y_test, y_pred))
  scores.append(score)

In [27]:
for estimator in [voting_clf, voting_clf2]:
  y_pred = estimator.predict(X_test)
  score = "{}: {:.2f}".format(estimator.__class__.__name__, accuracy(y_test, y_pred))
  scores.append(score)
print(scores)

['RandomForestClassifier: 0.96', 'ExtraTreesClassifier: 0.97', 'LinearSVC: 0.86', 'VotingClassifier: 0.97', 'VotingClassifier: 0.97']


We can see that our voting classifier performs slightly better than the individual classifiers (when scores are not rounded off).

2. Run the individual classifiers from the previous exercise to make predictions on the validation set, and create a new training set with the resulting predictions: each training instance is a vector containing the set of predictiosn from all your classifiers for an image, and the target is the image's class. Train a classifier on this new training set.

In [29]:
import numpy as np

In [30]:
x_val_predictions = np.empty((len(X_val), len(estimators)), dtype = np.float32)

for index, estimator in enumerate(estimators):
  x_val_predictions[:, index] = estimator.predict(X_val)

In [31]:
x_val_predictions

array([[5., 5., 5.],
       [8., 8., 8.],
       [2., 2., 2.],
       ...,
       [7., 7., 7.],
       [6., 6., 6.],
       [7., 7., 7.]], dtype=float32)

In [32]:
rnd_forest_blender = RandomForestClassifier(n_estimators = 200, oob_score=True,
                                            random_state=42)
rnd_forest_blender.fit(x_val_predictions, y_val)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

In [33]:
rnd_forest_blender.oob_score_

0.9702

We could fine-tune this blender or try other types of blender(eg. svm), then select the best one using cross-validation, as always.

Exercise: Congratulations, you have just trained a blender, and together with the classifiers they form a stacking ensemble! Now let's evaluate the ensemble on the test set. For each image in the test set, make predictions with all your classifiers, then feed the predictions to the blender to get the ensemble's predictions. How does it compare to the voting classifier you trained earlier?

In [34]:
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
  X_test_predictions[:, index] = estimator.predict(X_test)

In [35]:
X_test_predictions

array([[8., 8., 8.],
       [4., 4., 4.],
       [8., 8., 5.],
       ...,
       [3., 3., 3.],
       [8., 8., 5.],
       [3., 3., 3.]], dtype=float32)

In [36]:
y_pred = rnd_forest_blender.predict(X_test_predictions)

In [37]:
accuracy(y_test, y_pred)

0.9658

This stacking ensemble does not perform well as the voting classifier we trained earlier, its not quite good as the best individual classifier.